# Temporal Causal Inference

In this notebook, we will look at an example of causal effect inference for Temporal Dependencies within a Graphical Causal Model.  We demonstrate the use of temporal shift to normalise temporal datasets and establish causal effects.

Estimating the lag between the parent and action node is quite challenging in temporal causal inference. In this tutorial, we will assume we have the ground truth causal graph as an input.

In [ ]:
import networkx as nx
import pandas as pd
from dowhy.utils.timeseries import create_graph_from_csv,create_graph_from_user
from dowhy.utils.plotting import plot, pretty_print_graph

The user can create a csv file with the edges in the temporal graph. The columns in the csv are node_1, node_2, time_lag which represents an directed edge node_1 -> node_2 with the time lag of time_lag. Let us consider the following graph as the input

| node1  | node2  | time_lag |
|--------|--------|----------|
| V1     | V2     | 3        |
| V2     | V3     | 4        |
| V5     | V6     | 1        |
| V4     | V7     | 4        |
| V7     | V6     | 3        |

In [ ]:
# Input a csv file with the edges in the graph with the columns: node_1,node_2,time_lag
file_path = "../datasets/temporal_graph.csv"

# Create the graph from the CSV file
graph = create_graph_from_csv(file_path)
plot(graph)

## Dataset Shifting and Filtering

In temporal causal inference, accurately estimating causal effects often requires accounting for time lags between nodes in a graph. For instance, if node_1 influences node_2 with a time lag of 5 timestamps, we represent this dependency as node_1(t-5) -> node_2(t). To maintain this causal relationship in our analysis, we need to shift the columns by the given time lag.

When considering node_2 as the target node, the data for node_1 should be shifted down by 5 timestamps. This adjustment ensures that the new edge node_1(t) -> node_2(t) accurately represents the same lagged dependency as before. Shifting the data in this manner aligns the time series data correctly, allowing us to properly analyze and estimate the causal effects.

In [ ]:
from dowhy.timeseries.temporal_shift import find_lagged_parents,shift_columns_by_lag,_filter_columns

In [ ]:
# read the dataframe in a csv format from the user
dataset_path="../datasets/temporal_dataset.csv"
dataframe=pd.read_csv(dataset_path)

# the node for which effect estimation has to be done, node:6
target_node = 'V6'

# find the action nodes of the given target node with respective lag times
parents, time_lags = find_lagged_parents(graph, target_node)
print ("Parents of the target node with respective time lags: ", parents, time_lags)

In [ ]:
time_shifted_df = shift_columns_by_lag(dataframe,parents,time_lags,filter=True, child_node=target_node)
time_shifted_df.head()

## Cause Estimation using Dowhy

Once you have the new dataframe, causal effect estimation can be performed on the target node with respect to the action nodes.

In [ ]:
# perform causal effect estimation on this new dataset
import dowhy
from dowhy import CausalModel

model = CausalModel(
    data=time_shifted_df,
    treatment='V5',
    outcome='V6',
    proceed_when_unidentifiable=True  # Proceed even if the causal graph is not fully identifiable
)

identified_estimand = model.identify_effect()

estimate = model.estimate_effect(identified_estimand,
                                 method_name="backdoor.linear_regression",
                                 test_significance=True)

print(estimate)

In [ ]:
from dowhy.utils.timeseries import create_graph_from_dot_format

file_path = "../datasets/temporal_graph.dot"

graph = create_graph_from_dot_format(file_path)
plot(graph)


# Converting Tigramite Graph to a causal graph from the dataset

Tigramite is a popular temporal causal discovery library. In this section, we highlight how the causal graph can be obtained by applying PCMCI+ algorithm from tigramite to obtain a temporal causal graph

In [ ]:
import tigramite
from tigramite import data_processing
import matplotlib.pyplot as plt
import pandas as pd
# read the dataframe in a csv format from the user
dataset_path="../datasets/temporal_dataset.csv"
dataframe=pd.read_csv(dataset_path)
dataframe = dataframe.astype(float)
var_names = dataframe.columns
# convert the dataframe values to float
dataframe = tigramite.data_processing.DataFrame(dataframe.values, var_names=var_names)

In [ ]:
from tigramite import plotting as tp
tp.plot_timeseries(dataframe, figsize=(15, 5)); plt.show()

In [ ]:
from tigramite.pcmci import PCMCI
from tigramite.independence_tests.parcorr import ParCorr
import numpy as np
parcorr = ParCorr(significance='analytic')
pcmci = PCMCI(
    dataframe=dataframe, 
    cond_ind_test=parcorr,
    verbosity=1)

In [ ]:
correlations = pcmci.run_bivci(tau_max=3, val_only=True)['val_matrix']
matrix_lags = np.argmax(np.abs(correlations), axis=2)

In [ ]:
tau_max = 3
pc_alpha = None
pcmci.verbosity = 2

results = pcmci.run_pcmciplus(tau_min=0, tau_max=tau_max, pc_alpha=pc_alpha)

In [ ]:
from dowhy.utils.timeseries import create_graph_from_networkx_array

graph = create_graph_from_networkx_array(results['graph'], var_names)

plot(graph)